# CartPole-v0

> reference: 

A pole is attached by an un-actuated joint to a cart, which moves along a frictionless track. The system is controlled by applying a force of +1 or -1 to the cart. The pendulum starts upright, and the goal is to prevent it from falling over. 

A `reward` of +1 is provided for every timestep that the pole remains upright. The episode ends when `the pole is more than 15 degrees from vertical`, or `the cart moves more than 2.4 units from the center`.

In [9]:
import gym
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from keras.optimizers import Adam
from keras import backend as K
from keras.callbacks import History
from collections import deque
import itertools
import numpy as np
import random

random.seed(42)

In [10]:
class A2CAgent():
    def __init__(self, state_size, action_size):
        self._state_size = state_size
        self._action_size = action_size        
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95    # discount rate
        self.epsilon = 1.0  # exploration rate
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self._learning_rate = 0.001
        
        self.value_size = 1
        self.actor, self.critic = self._build_model()
        
    def _build_model(self, hidden_size=24):
        state = Input(batch_shape=(None,  self._state_size))
        shared = Dense(hidden_size, input_dim=self._state_size, activation='relu', kernel_initializer='glorot_uniform')(state)

        actor_hidden = Dense(hidden_size, activation='relu', kernel_initializer='glorot_uniform')(shared)
        action_prob = Dense(self._action_size, activation='softmax', kernel_initializer='glorot_uniform')(actor_hidden)
        actor = Model(inputs=state, outputs=action_prob)
        
        value_hidden = Dense(hidden_size, activation='relu', kernel_initializer='he_uniform')(shared)
        state_value = Dense(1, activation='linear', kernel_initializer='he_uniform')(value_hidden)
        critic = Model(inputs=state, outputs=state_value)

        opt = Adam(lr=self._learning_rate)
        actor.compile(loss='categorical_crossentropy', optimizer=opt)
        critic.compile(loss='mean_squared_error', optimizer=opt)
        
        actor.summary()
        critic.summary()

        return actor, critic
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

      
    # update policy network every episode
    def train_model(self, batch_size=20):
        def batch(iterable, n=1):
            l = len(iterable)
            for ndx in range(0, l, n):
                yield list(itertools.islice(iterable, ndx, min(ndx + n, l)))
        
        history = History()
        batch_data = list(batch(self.memory, batch_size))
        selected_batch = random.sample(batch_data, 1)
        
        for state, action, reward, next_state, done in selected_batch[0]:
            target = np.zeros((1, self.value_size))
            advantages = np.zeros((1, self._action_size))

            value = self.critic.predict(state)[0]
            next_value = self.critic.predict(next_state)[0]

            if done:
                advantages[0][action] = reward - value
                target[0][0] = reward
            else:
                advantages[0][action] = reward + self.gamma * (next_value) - value
                target[0][0] = reward + self.gamma * next_value

            self.actor.fit(state, advantages, epochs=1, verbose=0)
            self.critic.fit(state, target, epochs=1, verbose=0) 

    # using the output of policy network, pick action stochastically
    def takeAction(self, state):
        policy = self.actor.predict(state, batch_size=1).flatten()
        return np.random.choice(self._action_size, 1, p=policy)[0]


In [11]:
episodes = 500
render = False
shows = 20
total_reward = list()

env = gym.make('CartPole-v0')
state = env.reset()
agent = A2CAgent(state.shape[0], env.action_space.n)

for e in range(episodes):
    state = env.reset()
    state = np.reshape(state, [1, 4])
    
    for time_t in range(500):
        
        if render:
            env.render()
            
        action = agent.takeAction(state)
        next_state, reward, done, _ = env.step(action)
        
        next_state = np.reshape(next_state, [1, 4])
        agent.remember(state, action, reward, next_state, done)
        state = next_state
        
        if done:
            total_reward.append(time_t)
            break
            
    agent.train_model(32)
    
    if e % shows == 0:
        print("Episode: {}, Total Reward: {}".format(e, np.mean(total_reward[-shows:])))

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 4)                 0         
_________________________________________________________________
dense_4 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_5 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 4)       

/Users/yuting/yuting_data/github/openAI/.venv/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.119619). Check your callbacks.
  % delta_t_median)


Episode: 80, Total Reward: 10.95
Episode: 100, Total Reward: 10.1
Episode: 120, Total Reward: 33.75
Episode: 140, Total Reward: 37.25
Episode: 160, Total Reward: 70.45
Episode: 180, Total Reward: 81.1
Episode: 200, Total Reward: 39.75
Episode: 220, Total Reward: 69.75
Episode: 240, Total Reward: 95.5
Episode: 260, Total Reward: 45.35
Episode: 280, Total Reward: 74.0
Episode: 300, Total Reward: 101.2
Episode: 320, Total Reward: 133.0
Episode: 340, Total Reward: 156.15
Episode: 360, Total Reward: 167.4
Episode: 380, Total Reward: 100.6
Episode: 400, Total Reward: 134.4
Episode: 420, Total Reward: 30.45
Episode: 440, Total Reward: 150.55
Episode: 460, Total Reward: 172.15
Episode: 480, Total Reward: 169.45
